In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import pyspark
from pyspark.sql import SparkSession

import category_encoders as ce
try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [2]:
propiedadesTrain = pd.read_csv('train.csv')
propiedadesTest = pd.read_csv('test.csv')

In [3]:
propiedadesTrain.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,23533.0,NaN,NaN,2015-08-23 00:00:00,0.0,0.0,0.0,0.0,0.0,2273000.0
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,24514.0,19.310205,-99.227655,2013-06-28 00:00:00,0.0,0.0,0.0,1.0,1.0,3600000.0
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,48551.0,NaN,NaN,2015-10-17 00:00:00,0.0,0.0,0.0,0.0,0.0,1200000.0
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,53666.0,19.301890,-99.688015,2012-03-09 00:00:00,0.0,0.0,0.0,1.0,1.0,650000.0
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,47835.0,NaN,NaN,2016-06-07 00:00:00,0.0,0.0,0.0,0.0,0.0,1150000.0


In [4]:
propiedadesTest.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,metrostotales,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos
0,4941,"casa en venta en miguel hidalgo, distrito federal",<p>excelente casa estilo moderno.</p>,Casa,Bosque de Cedros,Miguel Hidalgo,Distrito Federal,29.0,3.0,NaN,...,NaN,NaN,19.408668,-99.246767,2013-07-20 00:00:00,0.0,0.0,0.0,0.0,0.0
1,51775,departamentos en venta en montebello,<p>departamento una recamara:\n</p><p>departam...,Apartamento,NaN,Mérida,Yucatán,NaN,1.0,1.0,...,67.0,113851.0,21.032480,-89.592424,2015-10-24 00:00:00,0.0,0.0,0.0,0.0,0.0
2,115253,departamento nuevo delegación coyoacán de 87 m...,"departamento nuevo de 87.06 m2, 1 cajón de est...",Apartamento,"Pueblo de los Reyes, Coyoacán, Mexico D.F.",Coyoacán,Distrito Federal,0.0,2.0,1.0,...,100.0,23620.0,19.332829,-99.152913,2015-05-30 00:00:00,0.0,0.0,0.0,0.0,1.0
3,299321,departamento en venta en acapulco,<p> raíces dv001 precioso departamento tipo k...,Apartamento,NaN,Acapulco de Juárez,Guerrero,2.0,2.0,2.0,...,86.0,129347.0,16.860487,-99.878383,2015-04-02 00:00:00,0.0,0.0,0.0,0.0,0.0
4,173570,bonita casa sola equipada de dos niveles en lo...,"<p>casa sola, bonita de dos rec&aacute;maras u...",Casa,CEDROS,Tultitlán,Edo. de México,10.0,2.0,1.0,...,76.0,57125.0,19.640482,-99.127273,2013-08-15 00:00:00,0.0,0.0,0.0,1.0,1.0


In [5]:
propiedadesTrain.columns

Index(['id', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio'],
      dtype='object')

# los Primeros features que tomo son cantidas de habitaciones, cantidad de baños, cantidad de garages, antiguedad, metrosTotales, metrosCubiertos, idZona, fecha, y si tiene gimnasio, sum, piscina, colegios cerca o centros comerciales cerca.

In [6]:
features = propiedadesTrain.loc[:,['id','provincia','ciudad','antiguedad', 'habitaciones', 'garages', 'banos',\
       'metroscubiertos', 'metrostotales', 'idzona','fecha',\
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',\
       'centroscomercialescercanos', 'precio']]

In [7]:
features.head()

,id,provincia,ciudad,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
0,254099,Distrito Federal,Benito Juárez,NaN,2.0,1.0,2.0,80.0,80.0,23533.0,2015-08-23 00:00:00,0.0,0.0,0.0,0.0,0.0,2273000.0
1,53461,Distrito Federal,La Magdalena Contreras,10.0,3.0,2.0,2.0,268.0,180.0,24514.0,2013-06-28 00:00:00,0.0,0.0,0.0,1.0,1.0,3600000.0
2,247984,Jalisco,Tonalá,5.0,3.0,2.0,2.0,144.0,166.0,48551.0,2015-10-17 00:00:00,0.0,0.0,0.0,0.0,0.0,1200000.0
3,209067,Edo. de México,Zinacantepec,1.0,2.0,1.0,1.0,63.0,67.0,53666.0,2012-03-09 00:00:00,0.0,0.0,0.0,1.0,1.0,650000.0
4,185997,Jalisco,Zapopan,10.0,2.0,1.0,1.0,95.0,95.0,47835.0,2016-06-07 00:00:00,0.0,0.0,0.0,0.0,0.0,1150000.0


In [8]:
#agrego columnas año, mes y dia
features['fecha_ok'] = pd.to_datetime(propiedadesTrain['fecha'])
features['anio'] = features['fecha_ok'].dt.year
features['mes'] = features['fecha_ok'].dt.month
features['dia'] = features['fecha_ok'].dt.day
features = features.drop(['fecha_ok','fecha'],axis=1)
features.head()

,id,provincia,ciudad,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,anio,mes,dia
0,254099,Distrito Federal,Benito Juárez,NaN,2.0,1.0,2.0,80.0,80.0,23533.0,0.0,0.0,0.0,0.0,0.0,2273000.0,2015,8,23
1,53461,Distrito Federal,La Magdalena Contreras,10.0,3.0,2.0,2.0,268.0,180.0,24514.0,0.0,0.0,0.0,1.0,1.0,3600000.0,2013,6,28
2,247984,Jalisco,Tonalá,5.0,3.0,2.0,2.0,144.0,166.0,48551.0,0.0,0.0,0.0,0.0,0.0,1200000.0,2015,10,17
3,209067,Edo. de México,Zinacantepec,1.0,2.0,1.0,1.0,63.0,67.0,53666.0,0.0,0.0,0.0,1.0,1.0,650000.0,2012,3,9
4,185997,Jalisco,Zapopan,10.0,2.0,1.0,1.0,95.0,95.0,47835.0,0.0,0.0,0.0,0.0,0.0,1150000.0,2016,6,7


# Agrego feature que es "dato bien cargado": si el dato tiene metros totales mayor o igual a metros cubiertos es 1, sino es 0.

In [14]:
features['datoBienCargado'] = (features['metrostotales'] >= features['metroscubiertos']).astype(int)
features['datoBienCargado'] = features['datoBienCargado'].fillna(0)

In [15]:
features.head()

,id,provincia,ciudad,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,anio,mes,dia,datoBienCargado
0,254099,Distrito Federal,Benito Juárez,NaN,2.0,1.0,2.0,80.0,80.0,23533.0,0.0,0.0,0.0,0.0,0.0,2273000.0,2015,8,23,1
1,53461,Distrito Federal,La Magdalena Contreras,10.0,3.0,2.0,2.0,268.0,180.0,24514.0,0.0,0.0,0.0,1.0,1.0,3600000.0,2013,6,28,0
2,247984,Jalisco,Tonalá,5.0,3.0,2.0,2.0,144.0,166.0,48551.0,0.0,0.0,0.0,0.0,0.0,1200000.0,2015,10,17,1
3,209067,Edo. de México,Zinacantepec,1.0,2.0,1.0,1.0,63.0,67.0,53666.0,0.0,0.0,0.0,1.0,1.0,650000.0,2012,3,9,1
4,185997,Jalisco,Zapopan,10.0,2.0,1.0,1.0,95.0,95.0,47835.0,0.0,0.0,0.0,0.0,0.0,1150000.0,2016,6,7,1


# Aplico un mean encoding a ciudad y provincia:

In [18]:
cat_ciudad_provincia = features.loc[:,['ciudad','provincia']]
target_enc = ce.TargetEncoder(cols=cat_ciudad_provincia)
target_enc.fit(features[cat_ciudad_provincia], features['datoBienCargado'])

ValueError: Must pass DataFrame with boolean values only